Solve a 1D BVP

Consider the following ODE (this ODE is based off of Moin Problem 4.26 and can represent the temperature distribution in a body whose cross-sectional area varies with position, but whose temperature variation in the lateral directions is negligible compared to the axial direction): 

$\begin{equation}
\frac{d^2T}{dx^2} + A(x)\frac{dT}{dx} + B(x)T = f(x)\quad {\rm for}\quad x\in [0,L]
\end{equation}$

with

$\begin{equation}
A(x) = -\frac{x+3}{x+1},\quad B(x) = \frac{x+3}{(x+1)^2}, \quad {\rm and} \quad f(x) = 2(x+1) + 3B(x)
\end{equation}$

Recall that using the direct method with $O(h^2)$ central difference schemes for each derivative term leads to a system of equations of the form: 

$\begin{equation}
\alpha_j T_{j+1} + \beta_j T_j + \gamma_j T_{j-1} = f_j, \quad j = 1, 2, ..., N-1
\end{equation}$

The expressions for $\alpha_j$, $\beta_j$ and $\gamma_j$ were derived in class (you do not need to repeat these derivations).

Consider the following mixed boundary conditions: 

$\begin{equation}
a_0y(0) + b_0y'(0) = c_0
\end{equation}$

and

$\begin{equation}
a_Ly(L) + b_Ly'(L) = c_L
\end{equation}$

Suppose we wish to approximate the derivative terms above using the following one-sided difference schemes: 

$\begin{equation}
y'(0) = \frac{-3y_0 + 4y_1 - y_2}{2h} + O(h^2)
\end{equation}$

and

$\begin{equation}
y'(L) = \frac{y_{N-2} - 4y_{N-1} + 3y_N}{2h} + O(h^2)
\end{equation}$

Write a function Tsolve(L, N, BCleft, BCright) that solves the ODE above using the specified second-order difference schemes for the mixed boundary conditions and constant grid spacing . The function must be able to handle general mixed boundary conditions at both ends of the domain. These are defined by the 1D arrays BCleft = [a0, b0, c0] and BCright = [aL, bL, cL]. Note that the special cases of Dirichlet or Neumann boundary conditions can be imposed by chosing appropriate values of coefficients ($e.g. b_0 = 0$ would impose a Dirichlet condition at the left end of the domain).

Your function must use the following as input:

| Name    | Type           | Description                                                                                               |
|---------|----------------|-----------------------------------------------------------------------------------------------------------|
| L       | float          | Domain length (domain is $x \in [0, L]$)                                                                  |
| N       | int            | Number of intervals to use; there are \$N + 1\$ gridpoints                                                |
| BCleft  | 1D numpy array | 1D numpy array of length 3, containing the coefficients $a_0$, $b_0$, and $c_0$: BCleft = [a0, b0, c0]    |
| BCright | 1D numpy array | 1D numpy array of length 3, containing the coefficients $a_L$, $b_L$, and $c_L$: BCright = [aL, bL, cL]   |

The graded function and description of required return variables are listed below:

| Name    | Type             | Description                                                                                               |
|-----------|-----------|--------------------------------------------------|
| Tsolve  |  python function | Function for finding the solve the given ODE                                                              |
| x       | 1D numpy array   | List of all of the \$N + 1\$ grid-points                                                                  |
| T       | 1D numpy array   | Solution computed by your function at all of the \$N + 1\$ grid-points                                    |
| BCright | 1D numpy array   | 1D numpy array of length 3, containing the coefficients $a_L$, $b_L$, and $c_L$: BCright = \[aL, bL, cL\] |

The function skeleton is provided below. The grader will test your function for different grid-sizes and boundary conditions to ensure your function is working correctly.

In [1]:
import numpy as np


def Tsolve(L, N, BCleft, BCright):
    # Step 1: Set Up the Grid
    x = np.linspace(0, L, N + 1)
    h = x[1] - x[0]

    # Step 2: Define A(x), B(x), and f(x)
    def A(x):
        return -(x + 3) / (x + 1)

    def B(x):
        return (x + 3) / ((x + 1) ** 2)

    def f(x):
        return 2 * (x + 1) + 3 * B(x)

    # Step 3: Discretize the ODE
    A_mat = np.zeros((N + 1, N + 1))
    b_vec = np.zeros(N + 1)

    # Interior points
    for j in range(1, N):
        xj = x[j]
        A_mat[j, j - 1] = 1 / h**2 - A(xj) / (2 * h)
        A_mat[j, j] = -2 / h**2 + B(xj)
        A_mat[j, j + 1] = 1 / h**2 + A(xj) / (2 * h)
        b_vec[j] = f(xj)

    # Step 4: Apply Boundary Conditions
    a0, b0, c0 = BCleft
    aL, bL, cL = BCright

    # Left boundary
    A_mat[0, 0] = a0 - 3 * b0 / (2 * h)
    A_mat[0, 1] = 4 * b0 / (2 * h)
    A_mat[0, 2] = -b0 / (2 * h)
    b_vec[0] = c0

    # Right boundary
    A_mat[N, N - 2] = bL / (2 * h)
    A_mat[N, N - 1] = -4 * bL / (2 * h)
    A_mat[N, N] = aL + 3 * bL / (2 * h)
    b_vec[N] = cL

    # Step 5: Solve the Linear System
    T = np.linalg.solve(A_mat, b_vec)

    return x, T

Solve the 1D convection-diffusion equation with forward Euler

The 1D convection-diffusion equation is

\begin{equation}
\frac{\partial T}{\partial t} = c \frac{\partial T}{\partial x} + \alpha \frac{\partial ^2T}{\partial x^2}\quad {\rm\ for\ } x\in[0,L], t\ge 0
\end{equation}

Write a function convdiff(L, Nx, dt, t_end, alpha, c) that solves the convection-diffusion PDE using forward Euler for time integration and the appropriate second order central difference schemes for the spatial derivatives. Your code function can assume the following initial profile:

\begin{equation}
T(x, 0) = \begin{cases}
			1 - (10x-1)^2 & \text{for $0 \le x \le 0.2$},\\
			0             & \text{for $0.2 < x \le 1$}
		\end{cases}
\end{equation}

The boundary conditions are homogeneous: $T(0, t) = T(1, t) = 0$

Your function must use the following as input:

| Name  | Type  | Description                                                 |
|-------|-------|-------------------------------------------------------------|
| L     | float | Domain length (domain is $x \in [0, L]$)                    |
| Nx    | int   | Number of intervals to use; there are $N_x + 1$ grid points |
| dt    | float | Time step size $\Delta t$                                   |
| t_end | float | Ending time for simulation                                  |
| alpha | float | coefficient $\alpha$ from the transient heat equation       |
| c     | float | convection coefficient $c$                                  |

The graded function and description of required return variables are listed below:

| Name     | Type            | Description                                                                                                      |
|----------|-----------------|------------------------------------------------------------------------------------------------------------------|
| convdiff | python function | Function to solve the convection-diffusion PDE with the forward Euler method and the specified initial condition |
| x        | 1D numpy array  | List of all the $N_x + 1$ grid points                                                                            |
| t        | 1D numpy array  | List of all the $N_t + 1$ time points in the simulation                                                          |
| T        | 2D numpy array  | T\[i, n\] stores the solution $T(x_i, t_n)$. The array is $(N_x + 1) \times (N_t + 1)$ in size                   |

The function skeleton is provided below. The grader will test your function for different grid-sizes and boundary conditions to ensure your function is working correctly.

import numpy as np

def convdiff(L, Nx, dt, t_end, alpha, c):

    return #x, t, T

In [ ]:
import numpy as np


def convdiff(L, Nx, dt, t_end, alpha, c):
    # Discretize the spatial domain
    dx = L / Nx
    x = np.linspace(0, L, Nx + 1)

    # Discretize the time domain
    Nt = int(t_end / dt)
    t = np.linspace(0, t_end, Nt + 1)

    # Initialize the temperature array
    T = np.zeros((Nx + 1, Nt + 1))

    # Apply initial condition
    for i in range(Nx + 1):
        if x[i] <= 0.2:
            T[i, 0] = 1 - (10 * x[i] - 1) ** 2
        else:
            T[i, 0] = 0

    # Time integration using Forward Euler
    for n in range(0, Nt):
        for i in range(1, Nx):
            dTdx = c * (T[i + 1, n] - T[i - 1, n]) / (2 * dx)
            d2Tdx2 = alpha * (T[i + 1, n] - 2 * T[i, n] + T[i - 1, n]) / (dx**2)
            T[i, n + 1] = T[i, n] + dt * (dTdx + d2Tdx2)

        # Apply boundary conditions
        T[0, n + 1] = 0
        T[Nx, n + 1] = 0

    return x, t, T

Crank-Nicolson for the 1D transient heat equation

Consider the 1D transient heat equation for a domain of length $L$:

\begin{equation}
\frac{\partial T}{\partial t} = \alpha \frac{\partial^2 T}{\partial x^2}\quad {\rm\ for\ } x\in[0,L], \quad t\ge 0
\end{equation}

Suppose we wish to use this equation to model heat transfer in a metallic bar that is insulated along its sides. The bar begins at a uniform temperature $T_{hot}$ At time $t=0$, the right end is exposed to air cooling via convection while the left end is maintained at a fixed temperature of $T_hot$. This situation corresponds to the following initial and boundary conditions:

\begin{equation}
T(x,0) = T_{hot} {\quad \rm\ for\ } x \in (0, L]
\end{equation}

\begin{equation}
T(0,t) = T_{hot}, \quad t\ge 0
\end{equation}

\begin{equation}
\frac{\partial T}{\partial x}(L,t) = \gamma(T_{\infty} - T(L,t)), \quad t\ge 0
\end{equation}

For the convection condition, $T_\infty$ represents the ambient air temperature and $\gamma$ represents the ratio of convection heat transfer coefficient to thermal conductivity, $\frac{h}{k}$.

Write a function heat1d_CN to implement the Crank-Nicolson scheme (trapezoid rule for time integration) for this problem. This scheme was derived in class, see class notes for the details and necessary equations.

Your function must use the following as input:

| Name  | Type  | Description                                                               |
|-------|-------|---------------------------------------------------------------------------|
| L     | float | domain length (domain is $x \in [0, L]$)                                  |
| Nx    | int   | number of intervals to use; there are $N_x + 1$ grid points               |
| dt    | float | time step size $\Delta t$                                                 |
| t_end | float | ending time for the simulation                                            |
| alpha | float | coefficient $\alpha$ from the transient heat equation                     |
| gamma | gloat | heat transfer coefficient $\gamma$ from the convection boundary condition |
| T_hot | float | $T_hot$ from the problem description                                      |
| T_inf | float | $T_\infty$ from the problem description                                   |


The graded function and description of required return variables are listed below:

| Name      | Type            | Description                                                                                    |
|-----------|-----------------|------------------------------------------------------------------------------------------------|
| heat1d_CN | python function | Function to implement the Crank-Nicholson scheme for this problem                              |
| x         | 1D numpy array  | list of all of the $N_x + 1$ grid points                                                       |
| t         | 1D numpy array  | list of all the $N_t + 1$ time points in the simulation                                        |
| T         | 2D numpy array  | T\[i, n\] stores the solution $T(x_i, t_n)$. The array is $(N_x + 1) \times (N_t + 1)$ in size |

The function skeleton is provided below. The grader will test your function for different grid-sizes and boundary conditions to ensure your function is working correctly.

import numpy as np

def heat1d_CN(L, Nx, dt, t_end, alpha, gamma, T_hot, T_inf):

    return #x, t, T